# CommaSeparatedListOutputParser 활용법

## 개요
LangChain의 `CommaSeparatedListOutputParser`는 LLM의 출력을 쉼표로 구분된 리스트 형태로 파싱하는 출력 파서입니다. 간단한 리스트 형태의 응답이 필요할 때 유용하게 사용할 수 있습니다.

### 주요 특징
- 쉼표로 구분된 문자열을 Python 리스트로 자동 변환
- 간단하고 직관적인 사용법
- 스트리밍 지원

## 환경 설정

In [1]:
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

True

## 기본 사용법

In [2]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

### 출력 형식 지침 확인

파서가 생성하는 형식 지침을 확인해보면, LLM에게 어떤 형태로 응답하라고 지시하는지 알 수 있습니다.

In [3]:
# 지침 확인
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


## 체인 구성 및 실행

In [4]:
# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    # 주제에 대한 다섯 가지를 나열하라는 템플릿
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],  # 입력 변수로 'subject' 사용
    # 부분 변수로 형식 지침 사용
    partial_variables={"format_instructions": format_instructions},
)

# ChatOpenAI 모델 초기화 (temperature=0으로 일관된 응답 유도)
model = ChatOpenAI(temperature=0)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

### 체인 실행 예제

In [5]:
# "대한민국 관광명소"에 대한 체인 호출 실행
result = chain.invoke({"subject": "대한민국 관광명소"})
result

['경복궁', '남산타워', '부산 해운대해수욕장', '제주도 성산일출봉', '경주 불국사']

## 스트리밍 지원

`CommaSeparatedListOutputParser`는 스트리밍도 지원합니다. 각 항목이 생성될 때마다 실시간으로 받아볼 수 있습니다.

In [6]:
# 스트림을 순회하며 각 항목을 실시간으로 출력
for s in chain.stream({"subject": "대한민국 관광명소"}):
    print(s)  # 스트림의 내용을 출력

['경복궁']
['남산타워']
['부산 해운대해수욕장']
['제주도']
['경주 불국사']


## 다양한 활용 예제

In [7]:
# 다양한 주제로 리스트 생성
subjects = ["프로그래밍 언어", "머신러닝 알고리즘", "데이터베이스 종류"]

for subject in subjects:
    result = chain.invoke({"subject": subject})
    print(f"{subject}: {result}")

프로그래밍 언어: ['Python', 'JavaScript', 'Java', 'C++', 'TypeScript']
머신러닝 알고리즘: ['선형 회귀', '로지스틱 회귀', '결정 트리', '랜덤 포레스트', '서포트 벡터 머신']
데이터베이스 종류: ['MySQL', 'PostgreSQL', 'MongoDB', 'Redis', 'Oracle']


## 커스텀 프롬프트 활용

In [8]:
# 더 복잡한 프롬프트 템플릿
custom_prompt = PromptTemplate(
    template="""당신은 {category} 전문가입니다.
{category}와 관련된 {number}개의 항목을 나열해주세요.
{format_instructions}""",
    input_variables=["category", "number"],
    partial_variables={"format_instructions": format_instructions},
)

# 새로운 체인 생성
custom_chain = custom_prompt | model | output_parser

# 실행
result = custom_chain.invoke({
    "category": "감정 표현",
    "number": "10"
})

print(f"긍정적인 단어들: {result}")

긍정적인 단어들: ['행복', '사랑', '희망', '평화', '감사', '기쁨', '성공', '자유', '열정', '조화']


## 오류 처리 및 검증

In [9]:
# 반환된 리스트의 길이 검증
def validate_list_length(subject, expected_count):
    prompt_with_count = PromptTemplate(
        template="List exactly {count} {subject}.\n{format_instructions}",
        input_variables=["subject", "count"],
        partial_variables={"format_instructions": format_instructions},
    )
    
    chain_with_count = prompt_with_count | model | output_parser
    
    result = chain_with_count.invoke({
        "subject": subject,
        "count": expected_count
    })
    
    print(f"요청한 항목 수: {expected_count}")
    print(f"실제 반환된 항목 수: {len(result)}")
    print(f"결과: {result}")
    
    return result

# 테스트
validate_list_length("프로그래밍 언어", 3)

요청한 항목 수: 3
실제 반환된 항목 수: 3
결과: ['파이썬', '자바스크립트', '자바']


## 요약

`CommaSeparatedListOutputParser`는 LangChain에서 제공하는 간단하면서도 유용한 출력 파서입니다.

### 주요 장점
1. **간단한 사용법**: 복잡한 설정 없이 바로 사용 가능
2. **자동 파싱**: 쉼표로 구분된 텍스트를 Python 리스트로 자동 변환
3. **스트리밍 지원**: 실시간으로 결과를 받아볼 수 있음
4. **유연성**: 다양한 프롬프트 템플릿과 함께 사용 가능

### 사용 시 주의사항
- LLM이 항상 정확한 개수의 항목을 반환하는 것은 아니므로, 필요시 검증 로직 추가
- 복잡한 구조의 데이터가 필요한 경우 `PydanticOutputParser` 등 다른 파서 고려